### Libraries Installations :

In [1]:
!pip install pandas numpy scikit-learn scipy matplotlib seaborn flask -q


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Reading Data :

In [ ]:
ratingsdf = pd.read_csv('''D:/github/Goodbooks_10k_Recommender_Project/Goodbooks_10k_Recommender_Project/Dataset/ratings.csv''')
booksdf = pd.read_csv('''D:/github/Goodbooks_10k_Recommender_Project/Goodbooks_10k_Recommender_Project/Dataset/books.csv''')
to_readdf = pd.read_csv(r'''D:/github/Goodbooks_10k_Recommender_Project/Goodbooks_10k_Recommender_Project/Dataset/to_read.csv''')
book_tagsdf = pd.read_csv(r'''D:/github/Goodbooks_10k_Recommender_Project/Goodbooks_10k_Recommender_Project/Dataset/book_tags.csv''')
tagsdf = pd.read_csv(r'''D:/github/Goodbooks_10k_Recommender_Project/Goodbooks_10k_Recommender_Project/Dataset/tags.csv''')

In [ ]:
print(ratingsdf.head(2))
print(booksdf.head(2))
print(to_readdf.head(2))
print(book_tagsdf.head(2))
print(tagsdf.head(2))

In [ ]:
# Check for missing values
ratingsdf.isnull().sum()

In [ ]:
booksdf.isnull().sum().sum()

In [ ]:
to_readdf.isnull().sum()

In [ ]:
book_tagsdf.isnull().sum()

In [ ]:
tagsdf.isnull().sum()

In [ ]:
booksdf['original_publication_year'] = booksdf['original_publication_year'].fillna(booksdf['original_publication_year'].mean())

In [ ]:
booksdf['original_title'] = booksdf['original_title'].fillna(booksdf['original_title'].mode()[0])

In [ ]:
booksdf['language_code'] = booksdf['language_code'].fillna('Unknown')

In [ ]:
# Merge ratings with books metadata on book_id
master_data = ratingsdf.merge(booksdf, on='book_id', how='left')


In [ ]:
master_data.head(3)

In [ ]:
# Merge book_tags with tags to get tag names
book_tagsdf = book_tagsdf.merge(tagsdf, on='tag_id', how='left')

In [ ]:
book_tagsdf.head(3)

In [ ]:
# Merge book_tags with tags to get tag names
book_tagsdf = book_tagsdf.merge(tagsdf, on='tag_id', how='left')


In [ ]:
book_tagsdf.head(3)

In [ ]:
# Aggregate tags for each book
tags_aggregated = book_tagsdf.groupby('goodreads_book_id')['tag_name'].apply(lambda x: ', '.join(x)).reset_index()

# Display the first few rows of the aggregated tags DataFrame
print(tags_aggregated.head())


In [ ]:
# Merge aggregated tags with the master_data
master_data = master_data.merge(tags_aggregated, left_on='goodreads_book_id', right_on='goodreads_book_id', how='left')

# Display the first few rows of the updated master DataFrame
print(master_data.head())


In [ ]:
# Save the master DataFrame to a CSV file for future analysis
master_data.to_csv(r'D:/github/Goodbooks_10k_Recommender_Project/Goodbooks_10k_Recommender_Project/Dataset/master_data.csv', index=False)
